In [1]:
print("Welcome")

Welcome


In [12]:
import requests
from bs4 import BeautifulSoup, SoupStrainer

import warnings
#warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

def get_list_of_questions(base_url, questions_link):
    #get page dom
    response = requests.get(questions_link)
    
    #https://www.crummy.com/software/BeautifulSoup/bs4/doc/
    soup = BeautifulSoup(response.content, "lxml",  parse_only=SoupStrainer("a", href=True))

    list = []
    for link in soup('a'):
        actualTag = link['href']
        if("problems" not in actualTag):
            continue

        if(actualTag.startswith("/")):
            actualTag = base_url + actualTag

        if(actualTag.endswith("/")):
            actualTag = actualTag[:-1]
            
        #print("\n ",actualTag)
        list.append(actualTag)
    
    return list
        
def main():        
    WEBSITE_BASE_URL = "https://www.codechef.com"
    QUESTIONS_LIST_URLS = ["https://www.codechef.com/problems/school",
                 "https://www.codechef.com/problems/easy", 
                 "https://www.codechef.com/problems/medium",
                 "https://www.codechef.com/problems/hard",
                 "https://www.codechef.com/problems/challenge"]
    
    list_of_problems = []
    for url in QUESTIONS_LIST_URLS:
        list_of_problems += get_list_of_questions(WEBSITE_BASE_URL, url)
      
    #removing extra links
    for url in QUESTIONS_LIST_URLS:
        list_of_problems = [value for value in list_of_problems if value != url]
        
    print(len(list_of_problems))
    #print(list_of_problems)

main()

1990
